In [26]:
# Don't be tempted - https://www.encyclopedia-titanica.org/titanic-deckplans/a-deck.html
# Some similar approaches - https://www.kaggle.com/code/ccastleberry/titanic-cabin-features

In [1]:
# Load necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Read Titanic Dataset
titanic_csv = r"C:\Data\IOD_Data\titanic_train.csv"
titanic = pd.read_csv(titanic_csv)

In [3]:
path2 = r"C:\Data\IOD_Data\titanic_test.csv"
titanic_test = pd.read_csv(path2)
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [4]:
# Want to concatanate the train and test datasets, this is for the data cleaning and manipulation purposes
titanic = pd.concat([titanic, titanic_test], ignore_index=True)

# Display the combined DataFrame
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Check DataTypes
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [6]:
# Checking nulll values for each column
titanic.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [7]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)

# Value counts of each cabin,
# Observation: Weird enough, I expected to see unique cabin entries. However, there are multiple cabin observations clumped together... But looking deeper into this, cabin numbers that logged in together have the same letter so we can still extract first letter from the cabin observations later on (these might be the cabins bought side by seid by families perhaps?)

titanic['Cabin'].value_counts()

Cabin
C23 C25 C27        6
G6                 5
B57 B59 B63 B66    5
C22 C26            4
F33                4
F2                 4
B96 B98            4
C78                4
F4                 4
D                  4
E34                3
B58 B60            3
A34                3
E101               3
C101               3
B51 B53 B55        3
C31                2
C55 C57            2
D37                2
C54                2
B35                2
C32                2
C7                 2
C124               2
E50                2
C6                 2
E44                2
C46                2
C92                2
D21                2
C116               2
C85                2
D20                2
B45                2
E8                 2
E121               2
E24                2
C62 C64            2
F G63              2
B20                2
B5                 2
B71                2
C126               2
D17                2
D19                2
B69                2
B41                2
C68    

In [8]:
# Want to explore the Sibling-Spouse column a little further, perhaps these are the families with multiple cabin observations
titanic['SibSp'].value_counts()

SibSp
0    891
1    319
2     42
4     22
3     20
8      9
5      6
Name: count, dtype: int64

In [9]:
# Subsetting the DataFrame where 'SibSp' is 1 or more
subset = titanic[titanic['SibSp'] >= 1][['Pclass', 'Name', 'Cabin', 'Age', 'SibSp']]

# Grouping by 'Pclass' and displaying the result
# grouped_subset = subset.groupby('Pclass').apply(lambda x: x)


# Displaying the subset grouped by 'Pclass'
grouped_subset = subset.sort_values('Pclass').reset_index(drop=True)
# Display the grouped subset
grouped_subset



,Pclass,Name,Cabin,Age,SibSp
0,1,"Hoyt, Mrs. Frederick Maxfield (Jane Anne Forby)",C93,35.00,1
1,1,"Bishop, Mrs. Dickinson H (Helen Walton)",B49,19.00,1
2,1,"Warren, Mr. Frank Manley",D37,64.00,1
3,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",C101,53.00,2
4,1,"Andrews, Miss. Kornelia Theodosia",D7,63.00,1
5,1,"Douglas, Mrs. Walter Donald (Mahala Dutton)",C86,48.00,1
6,1,"Spedden, Mr. Frederic Oakley",E34,45.00,1
7,1,"Taussig, Mr. Emil",E67,52.00,1
8,1,"Cornell, Mrs. Robert Clifford (Malvina Helen L...",C101,55.00,2
9,1,"Kenyon, Mr. Frederick R",D21,41.00,1


In [10]:
# Subsetting the DataFrame where 'SibSp' is 1 or more
subset2 = titanic[titanic['SibSp'] >= 1][['Pclass', 'Name', 'Cabin', 'Age', 'SibSp', 'Fare']]

# Counting the number of passengers in each cabin
subset2['Cabin_Count'] = subset2['Cabin'].map(subset2['Cabin'].value_counts())

# Sorting by 'Pclass' and 'Cabin'
grouped_subset = subset2.sort_values(['Pclass', 'Cabin']).reset_index(drop=True)

# Display the grouped subset
grouped_subset


,Pclass,Name,Cabin,Age,SibSp,Fare,Cabin_Count
0,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",A16,48.00,1,39.6000,1.0
1,1,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",A20,49.00,1,56.9292,1.0
2,1,"Dodge, Dr. Washington",A34,53.00,1,81.8583,2.0
3,1,"Dodge, Mrs. Washington (Ruth Vidaver)",A34,54.00,1,81.8583,2.0
4,1,"Dick, Mr. Albert Adrian",B20,31.00,1,57.0000,2.0
5,1,"Dick, Mrs. Albert Adrian (Vera Gillespie)",B20,17.00,1,57.0000,2.0
6,1,"Crosby, Capt. Edward Gifford",B22,70.00,1,71.0000,1.0
7,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",B26,64.00,1,26.5500,1.0
8,1,"Frolicher-Stehli, Mr. Maxmillian",B41,60.00,1,79.2000,2.0
9,1,"Frolicher-Stehli, Mrs. Maxmillian (Margaretha ...",B41,48.00,1,79.2000,2.0


In [11]:
# Making sure the dtype of this column is str so we can split the first letter later on
titanic['Cabin'] = titanic['Cabin'].astype(str)

# Extract the first letter of each cabin with list comprehesion, which would handle Nan values as well
titanic['Cabin_First_Letter'] = titanic['Cabin'].apply(lambda x: x[0] if pd.notna(x) and x != 'nan' else np.nan)

# Apply Function: We use the .apply() function to extract the first letter of each cabin. The lambda function checks if the value is not NaN and not the string 'nan' (which results from converting NaN to a string). It then extracts the first character.




# Result: The new column Cabin_First_Letter will contain the first letter of the cabin, or NaN if the original cabin value was missing.
titanic[['Cabin', 'Cabin_First_Letter', 'Pclass']]

,Cabin,Cabin_First_Letter,Pclass
0,nan,NaN,3
1,C85,C,1
2,nan,NaN,3
3,C123,C,1
4,nan,NaN,3
5,nan,NaN,3
6,E46,E,1
7,nan,NaN,3
8,nan,NaN,3
9,nan,NaN,2


In [12]:
titanic.groupby(['Pclass', 'Cabin_First_Letter']).agg({'Cabin_First_Letter' : 'count'})

Cabin_First_Letter
Pclass Cabin_First_Letter                    
1      A                                   22
       B                                   65
       C                                   94
       D                                   40
       E                                   34
       T                                    1
2      D                                    6
       E                                    4
       F                                   13
3      E                                    3
       F                                    8
       G                                    5

In [13]:
# Observation : 

# https://www.encyclopedia-titanica.org/titanic-deckplans/a-deck.html

# Looking at this ship map, it seems like there are more cabin A's than we have in the dataset. Also all A's are classified as first class, so it is almost impossible to get the most accurate info filled in to the missing cabin info.

In [14]:
# Would like to subset the data for only missing cabin value observations

cabin_nulls = titanic[titanic['Cabin_First_Letter'].isnull()]



In [15]:
# Subsetting the missing cabin numbers and grouping by the PClass
cabin_nulls.groupby(['Pclass']).agg({'Pclass' : 'count' , 'Fare' : 'mean'})

,Pclass,Fare
Pclass,,
1,67,70.268157
2,254,21.394537
3,693,13.351522


In [16]:
# I want to gather some payment info (min, max, mean, median of the Fare column per class and cabin letter)
# This info will help filling in missing cabin letters later on, it is just a gudie though

titanic.groupby(['Pclass', 'Cabin_First_Letter']).agg({'Cabin_First_Letter': 'count', 'Fare': ['mean', 'median', 'min', 'max']})

Cabin_First_Letter        Fare                     \
                                       count        mean    median      min   
Pclass Cabin_First_Letter                                                     
1      A                                  22   41.244314  35.07710   0.0000   
       B                                  65  122.383078  82.26670   0.0000   
       C                                  94  107.926598  86.28960  25.7000   
       D                                  40   58.919065  53.10000  25.9292   
       E                                  34   63.464706  55.44170  25.5875   
       T                                   1   35.500000  35.50000  35.5000   
2      D                                   6   13.595833  13.39585  12.8750   
       E                                   4   11.587500  11.42500  10.5000   
       F                                  13   23.423077  26.00000  10.5000   
3      E                                   3   11.000000  12.47500   8.0500   
       F                                   8    9.395837   7.65000   7.2292   
       G                                   5   14.205000  16.70000  10.4625   

                                     
                                max  
Pclass Cabin_First_Letter            
1      A                    81.8583  
       B                   512.3292  
       C                   263.0000  
       D                   113.2750  
       E                   134.5000  
       T                    35.5000  
2      D                    15.0458  
       E                    13.0000  
       F                    39.0000  
3      E                    12.4750  
       F                    22.3583  
       G                    16.7000

In [17]:
cabin_nulls.groupby(['Pclass']).agg({'Pclass': 'count', 'Fare': ['mean', 'median', 'min', 'max']})

Pclass       Fare                        
        count       mean   median  min       max
Pclass                                          
1          67  70.268157  39.6000  0.0  512.3292
2         254  21.394537  15.6646  0.0   73.5000
3         693  13.351522   8.0500  0.0   69.5500

In [18]:
# Fill missing 'Cabin_First_Letter' values for first-class passengers with a fare between 25 and 100 with 'D'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 1 & 
            (titanic['Fare'] >= 25) & (titanic['Fare'] <= 100), 
            'Cabin_First_Letter'] = "D"

# Fill missing 'Cabin_First_Letter' values for second-class passengers with a fare between 10 and 39 with 'F'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 2 & 
            (titanic['Fare'] >= 10) & (titanic['Fare'] <= 39), 
            'Cabin_First_Letter'] = "F"

# Fill missing 'Cabin_First_Letter' values for third-class passengers with a fare between 0 and 13 with 'E'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 3 & 
            (titanic['Fare'] >= 0) & (titanic['Fare'] <= 13), 
            'Cabin_First_Letter'] = "E"

# Fill missing 'Cabin_First_Letter' values for third-class passengers with a fare between 14 to 16 with 'G'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 3 & 
            (titanic['Fare'] > 13) & (titanic['Fare'] <= 16), 
            'Cabin_First_Letter'] = "G"

# Fill missing 'Cabin_First_Letter' values for third-class passengers with a fare between 16 to 23 with 'F'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 3 & 
            (titanic['Fare'] >= 16) & (titanic['Fare'] <= 23), 
            'Cabin_First_Letter'] = "F"


In [19]:
# After the above logical operations to fill in missing values, we still have a missing cabin letter observation
titanic.isnull().sum()

PassengerId             0
Survived              418
Pclass                  0
Name                    0
Sex                     0
Age                   263
SibSp                   0
Parch                   0
Ticket                  0
Fare                    1
Cabin                   0
Embarked                2
Cabin_First_Letter     31
dtype: int64

In [20]:
# From the above observation, we still have 38 missing Cabin letter value, so I am so keen to see what is going on
more_cabin_nulls = titanic[titanic['Cabin_First_Letter'].isnull()]
more_cabin_nulls

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_First_Letter
109,110,1.0,3,"Moran, Miss. Bertha",female,NaN,1,0,371110,24.1500,nan,Q,NaN
258,259,1.0,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,nan,C,NaN
301,302,1.0,3,"McCoy, Mr. Bernard",male,NaN,2,0,367226,23.2500,nan,Q,NaN
306,307,1.0,1,"Fleming, Miss. Margaret",female,NaN,0,0,17421,110.8833,nan,C,NaN
330,331,1.0,3,"McCoy, Miss. Agnes",female,NaN,2,0,367226,23.2500,nan,Q,NaN
334,335,1.0,1,"Frauenthal, Mrs. Henry William (Clara Heinshei...",female,NaN,1,0,PC 17611,133.6500,nan,S,NaN
373,374,0.0,1,"Ringhini, Mr. Sante",male,22.0,0,0,PC 17760,135.6333,nan,C,NaN
380,381,1.0,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,nan,C,NaN
419,420,0.0,3,"Van Impe, Miss. Catharina",female,10.0,0,2,345773,24.1500,nan,S,NaN
517,518,0.0,3,"Ryan, Mr. Patrick",male,NaN,0,0,371110,24.1500,nan,Q,NaN


In [21]:
more_cabin_nulls.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,31.000000,22.000000,31.000000,18.000000,31.000000,31.000000,30.000000
mean,694.387097,0.500000,2.096774,35.583333,0.741935,0.451613,102.502223
std,315.280370,0.511766,1.011759,13.215688,0.773207,0.767624,110.082892
min,110.000000,0.000000,1.000000,10.000000,0.000000,0.000000,23.250000
25%,400.500000,0.000000,1.000000,25.500000,0.000000,0.000000,24.150000
50%,709.000000,0.500000,3.000000,35.500000,1.000000,0.000000,24.150000
75%,913.000000,1.000000,3.000000,44.250000,1.000000,1.000000,151.550000
max,1267.000000,1.000000,3.000000,60.500000,2.000000,2.000000,512.329200


In [22]:
more_cabin_nulls.groupby(['Pclass']).agg({'Pclass': 'count', 'Fare': ['mean', 'median', 'min', 'max']})

Pclass        Fare                              
        count        mean     median      min       max
Pclass                                                 
1          14  192.440479  158.20835  106.425  512.3292
3          17   23.806250   24.15000   23.250   24.1500

In [23]:
# Fill missing 'Cabin_First_Letter' values for first-class passengers with a fare greater than or equal to 100 with 'A'
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 1 & 
            (titanic['Fare'] >= 100), 
            'Cabin_First_Letter'] = "A"


# Fill missing 'Cabin_First_Letter' values for third-class passengers with a greater than 23 with 'F' as it seems like it is the highest paying Pclass 3 cabin number
titanic.loc[titanic['Cabin_First_Letter'].isna() & titanic['Pclass'] == 3 & 
            (titanic['Fare'] > 23), 
            'Cabin_First_Letter'] = "F"

In [24]:
titanic.isnull().sum()

PassengerId             0
Survived              418
Pclass                  0
Name                    0
Sex                     0
Age                   263
SibSp                   0
Parch                   0
Ticket                  0
Fare                    1
Cabin                   0
Embarked                2
Cabin_First_Letter      1
dtype: int64

In [25]:
# Speechless
still_more_cabin_nulls = titanic[titanic['Cabin_First_Letter'].isnull()]
still_more_cabin_nulls

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_First_Letter
1043,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,nan,S,NaN
